POLITICAL BIAS

In [5]:
# Import necessary libraries
from google.colab import drive
import pandas as pd
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import torch

In [18]:
# Mount Google Drive and load the dataset
#drive.mount('/content/drive')
csv_file_path = 'data/processed/cleaned_articles.csv'
df = pd.read_csv(csv_file_path)

NameError: name 'drive' is not defined

In [9]:
# Display the first few rows of the dataframe to understand the data
df.head()

,title,cleaned_article
0,The symbols of Poland’s abortion protests expl...,The symbols of Polands abortion protests expla...
1,Impending Threat of Abortion Criminalization B...,Impending Threat of Abortion Criminalization B...
2,New lawsuits target state restrictions on abor...,New lawsuits target state restrictions on abor...
3,Indiana’s near-total abortion ban leads doctor...,Indianas near-total abortion ban leads doctors...
4,Legal challenge to Texas abortion law official...,Legal challenge to Texas abortion law official...


In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="bucketresearch/politicalBiasBERT")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
# Load model directly
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bucketresearch/politicalBiasBERT")
model = AutoModelForSequenceClassification.from_pretrained("bucketresearch/politicalBiasBERT")
model.eval()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [4]:
def score_article(article):
    # Tokenize the input article
    inputs = tokenizer(article, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Apply softmax to get probabilities
    probs = torch.softmax(outputs.logits, dim=1)

    # Get predicted class and probability
    predicted_class = torch.argmax(probs, dim=1).item()
    class_probabilities = probs.squeeze().tolist()

    return predicted_class, class_probabilities


In [16]:
# Initialize a list to store results
results = []

# Loop through articles and score them
for article in df['cleaned_article'].head(10).tolist():
    predicted_class, class_probabilities = score_article(article)

    # Append results to the list
      # [0] -> left
      # [1] -> center
      # [2] -> right
    results.append({
        "Article": article,
        "Predicted Class": predicted_class,
        "Class Probabilities": class_probabilities
    })

# Convert the results to a DataFrame
df = pd.DataFrame(results)

# Display the results in tabular format
print(df)


                                             Article  Predicted Class  \
0  The symbols of Polands abortion protests expla...                0   
1  Impending Threat of Abortion Criminalization B...                0   
2  New lawsuits target state restrictions on abor...                1   
3  Indianas near-total abortion ban leads doctors...                1   
4  Legal challenge to Texas abortion law official...                0   
5  Column Being pro-life has to be more than a sl...                1   
6  Frank Pavone cancels Mass but vows legal actio...                0   
7  Trump first president to attend anti-abortion ...                0   
8  Texas pro-life activist working with locals to...                0   
9  For Catholic supporters, Poland's abortion pro...                0   

                                 Class Probabilities  
0  [0.9791141152381897, 0.0178915373980999, 0.002...  
1  [0.7063356637954712, 0.2623268663883209, 0.031...  
2  [0.12690815329551697, 0.8642

RAG

In [10]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGr

In [1]:
from transformers import pipeline

# Assuming you have 10 articles stored in the 'articles' list
articles = df['cleaned_article'].head(10).tolist()

# Load the pipeline for text generation
pipe = pipeline("text-generation", model="google/datagemma-rag-27b-it")

# Define a function to check the hallucination score based on generated output
def check_hallucination(article):
    # Generate text for the article using the model
    generated_text = pipe(article, max_length=200, num_return_sequences=1)[0]['generated_text']

    # For a more advanced hallucination score, compare with actual facts or ask specific questions about the content
    # For simplicity, here we just output the generated text for review
    print(f"Original Article: {article[:100]}...")
    print(f"Generated Text: {generated_text[:100]}...")
    print("\n---\n")

# Loop through articles to generate text and check for hallucination
for article in articles:
    check_hallucination(article)


KeyboardInterrupt: 

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/datagemma-rag-27b-it")
model = AutoModelForCausalLM.from_pretrained("google/datagemma-rag-27b-it")

Hallucination

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="vectara/hallucination_evaluation_model", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

configuration_hhem_v2.py:   0%|          | 0.00/760 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vectara/hallucination_evaluation_model:
- configuration_hhem_v2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.
You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.


modeling_hhem_v2.py:   0%|          | 0.00/2.68k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
from transformers import AutoModelForSequenceClassification, PreTrainedTokenizerFast
import torch
# Load model directly
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("vectara/hallucination_evaluation_model", trust_remote_code=True)

You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [26]:
import pandas as pd
from transformers import AutoModelForSequenceClassification


# Article text
article_text = """
Following a recent constitutional court ruling that imposes a near-total ban on abortion in Poland, widespread protests erupted, marking the largest demonstrations since the fall of communism in 1989. The protests, primarily organized by the grassroots initiative Women’s Strike, are characterized by several significant symbols. The red lightning bolt, designed by Ola Jasionowska, serves as a warning against depriving women of their rights, although critics have attempted to associate it with Nazi insignia. The slogan "Wypierdala," meaning "Fuck off," has been prominently used, often alongside the iconic Solidarity logo. Other notable symbols include coat hangers, which highlight the dangers of illegal abortions, black umbrellas symbolizing mourning and resistance, and the anchor symbol representing Poland's fight against occupation, which has been adapted to signify "Polish Women Fighting." These symbols reflect the deep-seated anger towards the ruling Law and Justice party and the Catholic Church's influence over reproductive rights in Poland, showcasing the protestors' determination to defend women's rights.
"""

# Summary text
summary_text = """
Following a recent constitutional court ruling that imposes a near-total ban on abortion in Poland, widespread protests erupted, marking the largest demonstrations since the fall of communism in 1989. The protests, primarily organized by the grassroots initiative Women’s Strike, are characterized by several significant symbols, such as the red lightning bolt, the slogan "Wypierdala," coat hangers, black umbrellas, and an adapted anchor symbol. These symbols reflect the protestors' anger towards the ruling Law and Justice party and the Catholic Church's influence over reproductive rights, showcasing their determination to defend women's rights.
"""


# Define the test data, List[Tuple[str, str]]
pairs = [
    (article_text, summary_text)
]

# Step 1: Load the model
model = AutoModelForSequenceClassification.from_pretrained('vectara/hallucination_evaluation_model', trust_remote_code=True)

# Step 2: Use the model to predict
# Here you may need to transform pairs to the required input format if necessary
predictions = model.predict(pairs).numpy()  # Assuming predictions return an array-like structure

# Prepare the predictions (0 means not hallucinated, 1 means hallucinated)
# You might need to apply a threshold based on your model's outputs
# For demonstration, let's assume values close to 0.5 are considered hallucinated (use your model's logic)
predicted_labels = [1 if pred > 0.5 else 0 for pred in predictions]

# Create a DataFrame to display results in tabular format
results_df = pd.DataFrame({
   'Prediction': predicted_labels
})

# Display the results
print(results_df)


You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


   Prediction
0           1
